# RP3beta

## Import

In [1]:
## Allow more than one output for a single code cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
import pandas as pd
import scipy.sparse as sps
import numpy as np
import os

from skopt.space import Real, Integer, Categorical

## Set the numpy random seed
SEED = 42
np.random.seed(SEED)

os.getcwd()

'/home/alessio/Scrivania/RecSysChallenge2021'

In [3]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

from Evaluation.Evaluator import EvaluatorHoldout

from Recommenders.Recommender_import_list import *

from Recommenders.DataIO import DataIO

Tensorflow is not available


In [4]:
## Utility Functions
from Dataset.load_data import load_data
from Dataset.write_submission import write_submission
from Dataset.load_test_user_array import load_test_user_array

## Data Loading and Split

In [5]:
URM_all, ICM_dict = load_data()

In [24]:
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

URM_aug_train = sps.vstack([URM_train.copy().tocoo(), 
                            #ICM_dict['ICM_genre'].T.tocoo(),
                            ICM_dict['ICM_subgenre'].T.tocoo(), 
                            #ICM_dict['ICM_event'].T.tocoo(), 
                            ICM_dict['ICM_channel'].T.tocoo()], format='csr')

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10], exclude_seen = True)

EvaluatorHoldout: Ignoring 13644 ( 0.0%) Users that have less than 1 test interactions


In [25]:
URM_train_validaton, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validaton, train_percentage = 0.8)

URM_aug_train = sps.vstack([URM_train.copy().tocoo(), 
                            #ICM_dict['ICM_genre'].T.tocoo(),
                            ICM_dict['ICM_subgenre'].T.tocoo(), 
                            #ICM_dict['ICM_event'].T.tocoo(), 
                            ICM_dict['ICM_channel'].T.tocoo()], format='csr')

URM_aug_train_validation = sps.vstack([URM_train_validaton.copy().tocoo(), 
                            #ICM_dict['ICM_genre'].T.tocoo(),
                            ICM_dict['ICM_subgenre'].T.tocoo(), 
                            #ICM_dict['ICM_event'].T.tocoo(), 
                            ICM_dict['ICM_channel'].T.tocoo()], format='csr')

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10], exclude_seen = True)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10], exclude_seen = True)

EvaluatorHoldout: Ignoring 13637 ( 0.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 13644 ( 0.0%) Users that have less than 1 test interactions


In [7]:
test_UserID_array = load_test_user_array()

## Optimization

In [8]:
output_folder_path = "result_experiments/RP3beta_augmented_subgenre_channel/"

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data('noAugmentation' + RP3betaRecommender.RECOMMENDER_NAME + "_metadata.zip")

#search_metadata.keys()

hyperparameters_df = search_metadata["hyperparameters_df"]
result_on_validation_df = search_metadata["result_on_validation_df"]

hyperparameters_df['MAP'] = result_on_validation_df['MAP'].values.tolist()

threshold = 0.22

hyperparameters_df[hyperparameters_df['MAP'] > threshold]
print('Max value for the range: ', max(hyperparameters_df[hyperparameters_df['MAP'] > threshold].topK))
print('Min value for the range: ', min(hyperparameters_df[hyperparameters_df['MAP'] > threshold].topK))
print('Best MAP: ', max(hyperparameters_df['MAP']))
hyperparameters_df[max(hyperparameters_df['MAP']) == hyperparameters_df['MAP']]

,topK,alpha,beta,normalize_similarity,MAP
30,919,0.84241,0.701149,True,0.220853
31,800,1.0,0.715961,True,0.220984
32,800,1.0,0.688869,True,0.220909
33,800,0.847216,0.66878,True,0.220581
40,800,0.87384,0.719988,True,0.220264
46,1200,1.0,0.704682,True,0.220095
49,800,1.0,0.741612,True,0.220526


Max value for the range:  1200
Min value for the range:  800
Best MAP:  0.220984357297563


,topK,alpha,beta,normalize_similarity,MAP
31,800,1.0,0.715961,True,0.220984


In [9]:
output_folder_path = "result_experiments/RP3beta_augmented_subgenre_channel/"

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(RP3betaRecommender.RECOMMENDER_NAME + "_metadata.zip")

#search_metadata.keys()

hyperparameters_df = search_metadata["hyperparameters_df"]
result_on_validation_df = search_metadata["result_on_validation_df"]

hyperparameters_df['MAP'] = result_on_validation_df['MAP'].values.tolist()

threshold = 0.20

hyperparameters_df[hyperparameters_df['MAP'] > threshold]
print('Max value for the range: ', max(hyperparameters_df[hyperparameters_df['MAP'] > threshold].topK))
print('Min value for the range: ', min(hyperparameters_df[hyperparameters_df['MAP'] > threshold].topK))
print('Best MAP: ', max(hyperparameters_df['MAP']))
hyperparameters_df[max(hyperparameters_df['MAP']) == hyperparameters_df['MAP']]

,topK,alpha,beta,normalize_similarity,MAP
0,1009,0.599747,0.430482,False,0.202591
2,1783,0.676932,0.203678,True,0.201246
3,1183,0.659265,0.441199,True,0.210321
5,1914,0.358419,0.662776,True,0.210470
6,1562,0.198396,0.478845,True,0.206050
7,1638,0.101867,0.532748,True,0.205235
8,1227,0.443418,0.576483,True,0.212681
11,1579,0.836927,0.461486,False,0.206912
15,1167,0.098408,0.620544,True,0.206241
16,1971,0.366584,0.535616,True,0.208468


Max value for the range:  2000
Min value for the range:  1000
Best MAP:  0.22097109358819678


,topK,alpha,beta,normalize_similarity,MAP
46,1000,0.885538,0.71292,True,0.220971


In [10]:
output_folder_path = "result_experiments/RP3beta_augmented_subgenre_channel/"

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data('improvedRange' + RP3betaRecommender.RECOMMENDER_NAME + "_metadata.zip") # range topK from 800 to 1200

#search_metadata.keys()

hyperparameters_df = search_metadata["hyperparameters_df"]
result_on_validation_df = search_metadata["result_on_validation_df"]

hyperparameters_df['MAP'] = result_on_validation_df['MAP'].values.tolist()

threshold = 0.20

hyperparameters_df[hyperparameters_df['MAP'] > threshold]
print('Max value for the range: ', max(hyperparameters_df[hyperparameters_df['MAP'] > threshold].topK))
print('Min value for the range: ', min(hyperparameters_df[hyperparameters_df['MAP'] > threshold].topK))
print('Best MAP: ', max(hyperparameters_df['MAP']))
hyperparameters_df[max(hyperparameters_df['MAP']) == hyperparameters_df['MAP']]

,topK,alpha,beta,normalize_similarity,MAP
1,1144,0.421889,0.643489,True,0.214956
2,1131,1.166359,0.109542,True,0.203620
7,1172,0.179832,0.211472,True,0.200842
8,880,0.321892,0.579206,True,0.213856
9,1145,0.389597,0.229685,True,0.202370
11,1089,0.5176,0.430875,True,0.210034
12,1125,0.601994,0.194154,True,0.201812
13,1185,0.391639,0.236078,True,0.202702
14,959,0.367801,0.532778,True,0.212835
16,1125,0.765781,0.770872,True,0.215214


Max value for the range:  1200
Min value for the range:  800
Best MAP:  0.2199825345644639


,topK,alpha,beta,normalize_similarity,MAP
29,800,0.886577,0.68149,True,0.219983


In [11]:
output_folder_path = "result_experiments/RP3beta_augmented_subgenre_channel/"

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data('improvedRange3' + RP3betaRecommender.RECOMMENDER_NAME + "_metadata.zip") # range topK from 5 to 200

#search_metadata.keys()

hyperparameters_df = search_metadata["hyperparameters_df"]
result_on_validation_df = search_metadata["result_on_validation_df"]

hyperparameters_df['MAP'] = result_on_validation_df['MAP'].values.tolist()

threshold = 0.226

hyperparameters_df[hyperparameters_df['MAP'] > threshold]
print('Max value for the range: ', max(hyperparameters_df[hyperparameters_df['MAP'] > threshold].topK))
print('Min value for the range: ', min(hyperparameters_df[hyperparameters_df['MAP'] > threshold].topK))
print('Best MAP: ', max(hyperparameters_df['MAP']))
hyperparameters_df[max(hyperparameters_df['MAP']) == hyperparameters_df['MAP']]

,topK,alpha,beta,normalize_similarity,MAP
70,66,0.582972,0.54985,True,0.226040
85,51,0.0,0.571013,True,0.226162
89,61,0.233192,0.556471,True,0.226400
95,59,0.384235,0.568418,True,0.226833
96,61,0.664183,0.580731,True,0.226434
97,59,0.403549,0.580985,True,0.226368
98,59,0.52439,0.567004,True,0.226632


Max value for the range:  66
Min value for the range:  51
Best MAP:  0.22683304485993885


,topK,alpha,beta,normalize_similarity,MAP
95,59,0.384235,0.568418,True,0.226833


In [12]:
output_folder_path = "result_experiments/RP3beta_RP3beta_augmented_subgenre_channel_topK_categorical/"

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(RP3betaRecommender.RECOMMENDER_NAME + "_metadata.zip") # topK in [51, 59, 61, 66]

#search_metadata.keys()

hyperparameters_df = search_metadata["hyperparameters_df"]
result_on_validation_df = search_metadata["result_on_validation_df"]

hyperparameters_df['MAP'] = result_on_validation_df['MAP'].values.tolist()

threshold = 0.226

hyperparameters_df[hyperparameters_df['MAP'] > threshold]
print('Max value for the range: ', max(hyperparameters_df[hyperparameters_df['MAP'] > threshold].topK))
print('Min value for the range: ', min(hyperparameters_df[hyperparameters_df['MAP'] > threshold].topK))
print('Best MAP: ', max(hyperparameters_df['MAP']))
hyperparameters_df[max(hyperparameters_df['MAP']) == hyperparameters_df['MAP']]

,topK,alpha,beta,normalize_similarity,MAP
22,51,0.0,0.569896,True,0.226068
26,66,0.366369,0.556151,True,0.226108
35,51,0.512987,0.559573,True,0.226513
37,66,0.663584,0.551486,True,0.226208
39,51,0.302465,0.567289,True,0.226475
40,51,0.484685,0.554349,True,0.226535
41,51,0.550707,0.564618,True,0.226825
42,51,0.598138,0.566157,True,0.226762
43,51,0.60272,0.566298,True,0.226840
44,51,0.620106,0.566617,True,0.227149


Max value for the range:  66
Min value for the range:  51
Best MAP:  0.22714933325423442


,topK,alpha,beta,normalize_similarity,MAP
44,51,0.620106,0.566617,True,0.227149


In [20]:
output_folder_path = "result_experiments/RP3beta_RP3beta_augmented_subgenre_channel_topK_categorical2/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

n_cases = 10  # 50 with 30% random is a good number
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

In [21]:
from functools import partial
import os, multiprocessing

from HyperparameterTuning.run_hyperparameter_search import runHyperparameterSearch_Collaborative

runHyperparameterSearch_Collaborative(RP3betaRecommender,
                                      URM_train = URM_aug_train,
                                      URM_train_last_test = URM_train_validaton,
                                      metric_to_optimize = metric_to_optimize,
                                      cutoff_to_optimize = cutoff_to_optimize,
                                      n_cases = n_cases,
                                      n_random_starts = n_random_starts,
                                      evaluator_validation_earlystopping = evaluator_validation,
                                      evaluator_validation = evaluator_validation,
                                      evaluator_test = evaluator_test,
                                      output_folder_path = output_folder_path,
                                      resume_from_saved = True,
                                      similarity_type_list = None,
                                      parallelizeKNN = True)

SearchBayesianSkopt: Extending previous number of cases from 5 to 10.

SearchBayesianSkopt: Resuming 'RP3betaRecommender'... Loaded 1 configurations.
Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0005
Function value obtained: -0.1085
Current minimum: -0.1085
Iteration No: 2 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 61, 'alpha': 0.07237367126916518, 'beta': 0.8078719077622207, 'normalize_similarity': True}
EvaluatorHoldout: Processed 13634 (100.0%) in 12.57 sec. Users per second: 1084
SearchBayesianSkopt: Config 1 is suboptimal. Config: {'topK': 61, 'alpha': 0.07237367126916518, 'beta': 0.8078719077622207, 'normalize_similarity': True} - results: PRECISION: 0.1752824, PRECISION_RECALL_MIN_DEN: 0.1764674, RECALL: 0.0367325, MAP: 0.0792796, MAP_MIN_DEN: 0.0797079, MRR: 0.3678364, NDCG: 0.1802388, F1: 0.0607369, HIT_RATE: 0.8042394, ARHR_ALL_HITS: 0.537608

In [22]:
output_folder_path = "result_experiments/RP3beta_RP3beta_augmented_subgenre_channel_topK_categorical2/"

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(RP3betaRecommender.RECOMMENDER_NAME + "_metadata.zip") # topK in [51, 59, 61, 66]

#search_metadata.keys()

hyperparameters_df = search_metadata["hyperparameters_df"]
result_on_validation_df = search_metadata["result_on_validation_df"]

hyperparameters_df['MAP'] = result_on_validation_df['MAP'].values.tolist()

threshold = 0.0

hyperparameters_df[hyperparameters_df['MAP'] > threshold]
print('Max value for the range: ', max(hyperparameters_df[hyperparameters_df['MAP'] > threshold].topK))
print('Min value for the range: ', min(hyperparameters_df[hyperparameters_df['MAP'] > threshold].topK))
print('Best MAP: ', max(hyperparameters_df['MAP']))
hyperparameters_df[max(hyperparameters_df['MAP']) == hyperparameters_df['MAP']]

,topK,alpha,beta,normalize_similarity,MAP
0,66,0.379686,0.595717,True,0.108493
1,61,0.072374,0.807872,True,0.079280
2,61,0.073163,0.037224,True,0.066603
3,59,0.370322,0.884437,True,0.044755
4,66,1.0,1.0,True,0.000809
5,51,0.034692,0.429454,True,0.107142
6,66,1.0,0.37003,True,0.102244
7,51,1.0,0.675842,True,0.105345
8,51,0.028513,0.211611,True,0.086574
9,66,0.964218,0.508601,True,0.108498


Max value for the range:  66
Min value for the range:  51
Best MAP:  0.10849759355113645


,topK,alpha,beta,normalize_similarity,MAP
9,66,0.964218,0.508601,True,0.108498


In [28]:
RP3beta = RP3betaRecommender(URM_aug_train_validation)
RP3beta.fit(topK = 59, alpha = 0.384235, beta =	0.568418, normalize_similarity = True)
	
results_run_1, results_run_string_1 = evaluator_test.evaluateRecommender(RP3beta)

index = results_run_string_1.find("PRECISION:")
print(results_run_string_1[index:index+20])

index = results_run_string_1.find("MAP:")
print(results_run_string_1[index:index+14])

EvaluatorHoldout: Processed 13644 (100.0%) in 19.90 sec. Users per second: 686
PRECISION: 0.3670405
MAP: 0.2239100
